Concatenated surfaces saved at /home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/group_{hemi}_{para}.mgh

In [ ]:
import pandas as pd
import os
import subprocess
import shutil
from pathlib import Path

def setup_freesurfer():
    """Initialize FreeSurfer environment"""
    freesurfer_home = '/home/localadmin/freesurfer'
    os.environ['FREESURFER_HOME'] = freesurfer_home
    os.environ['SUBJECTS_DIR'] = "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia"
    
    setup_cmd = f"bash -c 'source {freesurfer_home}/SetUpFreeSurfer.sh; env'"
    try:
        process = subprocess.Popen(setup_cmd, stdout=subprocess.PIPE, shell=True)
        output, _ = process.communicate()
        
        for line in output.decode().split('\n'):
            if '=' in line:
                key, value = line.split('=', 1)
                os.environ[key] = value
        
        print("FreeSurfer environment initialized successfully")
    except Exception as e:
        print(f"Error setting up FreeSurfer: {e}")
        raise

def run_preprocessing(csv_path, parameters, hemispheres, output_dir):
    """Run mris_preproc for all parameters and hemispheres"""
    try:
        # Initialize FreeSurfer
        setup_freesurfer()
        subjects_dir = os.environ['SUBJECTS_DIR']
        
        # Create output directory
        os.makedirs(output_dir, exist_ok=True)
        
        # Read subject IDs
        df = pd.read_csv(csv_path)
        subjects = df['sub'].apply(lambda x: f'sub-CHUV{x:>03}').unique()
        
        
        # Full path to mris_preproc
        mris_preproc_path = os.path.join(os.environ['FREESURFER_HOME'], 'bin', 'mris_preproc')
        
        # Process each combination
        for hemi in hemispheres:
            for param in parameters:
                print(f"\nProcessing {param} for {hemi}")
                
                # Make output path relative to SUBJECTS_DIR
                rel_output = os.path.relpath(
                    os.path.join(output_dir, f'group_{hemi}_{param}.mgh'),  ##format
                    subjects_dir
                )
                
                # Build command with input files and their registration surfaces
                cmd = [
                    mris_preproc_path,
                    '--target', 'fsaverage',
                    '--hemi', hemi,
                    '--out', rel_output,
                    '--f', os.path.relpath('/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis/subject_list.txt', subjects_dir),
                ]
                
                # Add each subject's data and corresponding registration surface
                for subject in subjects:
                    # Create relative paths
                    pattern_file = os.path.join(
                        f"{subject}",
                        "surf",
                        f"{hemi}.{param}"
                    )
                    
                    reg_surface = os.path.join(
                        f"{subject}",
                        "surf",
                        f"{hemi}.sphere.reg"
                    )
                    
                    # Check if files exist using absolute paths
                    full_pattern_path = os.path.join(subjects_dir, pattern_file)
                    full_reg_path = os.path.join(subjects_dir, reg_surface)
                    
                    if os.path.isfile(full_pattern_path) and os.path.isfile(full_reg_path):
                        cmd.extend(['--is', full_pattern_path, '--surfreg', 'sphere.reg'])
                        print(f"Subject: {subject}")
                        print(f"Pattern file (relative): {pattern_file}")
                        print(f"Registration surface (relative): {reg_surface}")
                    else:
                        print(f"Missing files for subject {subject}. Pattern or registration surface not found.")
                        if not os.path.isfile(full_pattern_path):
                            print(f"Missing pattern file: {full_pattern_path}")
                        if not os.path.isfile(full_reg_path):
                            print(f"Missing registration file: {full_reg_path}")
                
                # Print the full command before execution for debugging
                print(f"\nFull Command for {param} {hemi}:\n{' '.join(cmd)}\n")
                
                # Execute command
                try:
                    print(f"Running command for {param} {hemi}...")
                    print("Command:", ' '.join(cmd))
                    process = subprocess.run(
                        cmd,
                        check=True,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE,
                        text=True,
                        env=os.environ
                    )
                    print(f"Successfully processed {param} for {hemi}")
                    if process.stdout:
                        print("Output:", process.stdout)
                    
                except subprocess.CalledProcessError as e:
                    print(f"Error processing {param} for {hemi}:")
                    print(f"Command failed with exit status {e.returncode}")
                    print(f"Error output:\n{e.stderr}")
                    print(f"Standard output:\n{e.stdout}")
                    
    except Exception as e:
        print(f"An error occurred during preprocessing: {e}")
        raise

if __name__ == "__main__":
    # Configuration
    CONFIG = {
        'csv_path': "/home/localadmin/hpc_mount/wangqiaochu/completed.csv",
        'parameters': ["thickness", "rd", "ad", "md", "fa", "ak", "rk", "mk"],
        'hemispheres': ["lh", "rh"],
        'output_dir': "/home/localadmin/hpc_mount/Cortical_Microstructure_Changes_in_Schizophrenia/new_group_analysis"
    }
    
    try:
        run_preprocessing(
            csv_path=CONFIG['csv_path'],
            parameters=CONFIG['parameters'],
            hemispheres=CONFIG['hemispheres'],
            output_dir=CONFIG['output_dir']
        )
    except Exception as e:
        print(f"Processing failed: {e}")